# Прогнозирование температуры плавления стали

**Описание проекта**

Чтобы оптимизировать производственные расходы, металлургический комбинат ООО «Так закаляем сталь» решил уменьшить потребление электроэнергии на этапе обработки стали. Вам предстоит построить модель, которая предскажет температуру стали.

**Описание этапа обработки стали**

Сталь обрабатывают в металлическом ковше вместимостью около 100 тонн. Чтобы ковш выдерживал высокие температуры, изнутри его облицовывают огнеупорным кирпичом. Расплавленную сталь заливают в ковш и подогревают до нужной температуры графитовыми электродами. Они установлены в крышке ковша.

Из сплава выводится сера (десульфурация), добавлением примесей корректируется химический состав и отбираются пробы. Сталь легируют — изменяют её состав — подавая куски сплава из бункера для сыпучих материалов или проволоку через специальный трайб-аппарат (англ. tribe, «масса»).

Перед тем как первый раз ввести легирующие добавки, измеряют температуру стали и производят её химический анализ. Потом температуру на несколько минут повышают, добавляют легирующие материалы и продувают сплав инертным газом. Затем его перемешивают и снова проводят измерения. Такой цикл повторяется до достижения целевого химического состава и оптимальной температуры плавки.

Тогда расплавленная сталь отправляется на доводку металла или поступает в машину непрерывной разливки. Оттуда готовый продукт выходит в виде заготовок-слябов (англ. slab, «плита»).

**Описание данных**

Данные состоят из файлов, полученных из разных источников:

1. `data_arc.csv` — данные об электродах;
2. `data_bulk.csv` — данные о подаче сыпучих материалов (объём);
3. `data_bulk_time.csv` — данные о подаче сыпучих материалов (время);
4. `data_gas.csv` — данные о продувке сплава газом;
5. `data_temp.csv` — результаты измерения температуры;
6. `data_wire.csv` — данные о проволочных материалах (объём);
7. `data_wire_time.csv` — данные о проволочных материалах (время).

Во всех файлах столбец `key` содержит номер партии. В файлах может быть несколько строк с одинаковым значением `key`: они соответствуют разным итерациям обработки.

**План работы**

1. **Предобработка и исследовательский анализ данных**. Проверить наличие пропусков, некорректных значений, дубликатов, провести их обработку, привести данные к нужному типу. Изучить признаки, их основные статистики, построить графики при необходимости. Удалить признаки, не несущие смысловой нагрузки или явлющиеся утечкой таргета. Подготовить итоговую таблицу "объектов-признаков", где объектами являются уникальные ключи партий.
   
2. **Обучение моделей**. Разделить выборку на обучающую и тестовую. Исследовать разные ML модели. Выбрать признаковое пространство, модель и ее гиперпараметры, имеющее лучшее качество на кросс-валидации
   
3. **Оценка эффективности моделей на тестовой выборке**. При недостижении целевого значения метрики качества вернуться к этапу 2, сформировать дополнительные признаки, продолжить выбор моделей и настройку гиперпараметров.

In [1]:
import pandas as pd
import numpy as np
import optuna
import plotly.express as px

from caseconverter import snakecase
from collections import defaultdict
from IPython.display import display

from ydata_profiling import ProfileReport
from fast_ml import eda

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from category_encoders import MEstimateEncoder

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error

In [2]:
FIG_WIDTH = 9 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42

In [3]:
try:
    raw = {
        'arc': pd.read_csv('data_arc.csv', parse_dates=['Начало нагрева дугой', 'Конец нагрева дугой']),
        'bulk_t': pd.read_csv('data_bulk_time.csv', parse_dates=
            ['Bulk 1',  'Bulk 2',  'Bulk 3',  'Bulk 4',  'Bulk 5',
             'Bulk 6',  'Bulk 7',  'Bulk 8',  'Bulk 9',  'Bulk 10',
             'Bulk 11', 'Bulk 12', 'Bulk 13', 'Bulk 14', 'Bulk 15']
        ),
        'bulk': pd.read_csv('data_bulk.csv'),
        'gas': pd.read_csv('data_gas.csv'),
        'temp': pd.read_csv('data_temp.csv', parse_dates=['Время замера']),
        'wire_t': pd.read_csv('data_wire_time.csv', parse_dates=
            ['Wire 1', 'Wire 2', 'Wire 3', 'Wire 4', 'Wire 5', 'Wire 6', 'Wire 7', 'Wire 8', 'Wire 9']
        ),
        'wire': pd.read_csv('data_wire.csv'),
    }
except:
    raw = {
        'arc': pd.read_csv('/datasets/data_arc.csv', parse_dates=['Начало нагрева дугой', 'Конец нагрева дугой']),
        'bulk_t': pd.read_csv('/datasets/data_bulk_time.csv', parse_dates=
            ['Bulk 1',  'Bulk 2',  'Bulk 3',  'Bulk 4',  'Bulk 5',
             'Bulk 6',  'Bulk 7',  'Bulk 8',  'Bulk 9',  'Bulk 10',
             'Bulk 11', 'Bulk 12', 'Bulk 13', 'Bulk 14', 'Bulk 15']
        ),
        'bulk': pd.read_csv('/datasets/data_bulk.csv'),
        'gas': pd.read_csv('/datasets/data_gas.csv'),
        'temp': pd.read_csv('/datasets/data_temp.csv', parse_dates=['Время замера']),
        'wire_t': pd.read_csv('/datasets/data_wire_time.csv', parse_dates=
            ['Wire 1', 'Wire 2', 'Wire 3', 'Wire 4', 'Wire 5', 'Wire 6', 'Wire 7', 'Wire 8', 'Wire 9']
        ),
        'wire': pd.read_csv('/datasets/data_wire.csv'),
    }

## Исследовательский анализ данных

Изучим основные зависимости в данных перед тем, как мы будем использовать их в алгоритмах машинного обучения.

Таблица-резюме:

In [4]:
for dataset in raw.keys():
    print('#' * 50, 'Summary for', dataset, 'dataset', '#' * 50)
    display(eda.df_info(raw[dataset]))

################################################## Summary for arc dataset ##################################################


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3214,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Начало нагрева дугой,datetime64[ns],DateTime,14875,"[1556881334000000000, 1556881648000000000, 155...",0,0.0
Конец нагрева дугой,datetime64[ns],DateTime,14876,"[1556881562000000000, 1556881833000000000, 155...",0,0.0
Активная мощность,float64,Numerical,14869,"[0.9760587506, 0.8056070805, 0.7443634272, 1.6...",0,0.0
Реактивная мощность,float64,Numerical,14866,"[0.6870837212, 0.5202852488, 0.4988054037, 1.0...",0,0.0


################################################## Summary for bulk_t dataset ##################################################


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3129,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Bulk 1,datetime64[ns],DateTime,252,"[None, 1556905819000000000, 155705095300000000...",2877,91.946309
Bulk 2,datetime64[ns],DateTime,22,"[None, 1557218179000000000, 155722353500000000...",3107,99.2969
Bulk 3,datetime64[ns],DateTime,1298,"[None, 1556914416000000000, 155694163000000000...",1831,58.517098
Bulk 4,datetime64[ns],DateTime,1014,"[1556882490000000000, 1556883998000000000, 155...",2115,67.59348
Bulk 5,datetime64[ns],DateTime,77,"[None, 1557218179000000000, 155722353500000000...",3052,97.53915
Bulk 6,datetime64[ns],DateTime,576,"[None, 1556909577000000000, 155691812000000000...",2553,81.591563
Bulk 7,datetime64[ns],DateTime,25,"[None, 1557229062000000000, 155791295300000000...",3104,99.201023
Bulk 8,datetime64[ns],DateTime,1,"[None, 1562348771000000000]",3128,99.968041
Bulk 9,datetime64[ns],DateTime,19,"[None, 1557792296000000000, 155779426700000000...",3110,99.392777


################################################## Summary for bulk dataset ##################################################


...

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3216,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Время замера,datetime64[ns],DateTime,15907,"[1556882178000000000, 1556882753000000000, 155...",0,0.0
Температура,float64,Numerical,175,"[1571.0, 1604.0, 1618.0, 1601.0, 1613.0, 1581....",2901,18.237254


################################################## Summary for wire_t dataset ##################################################


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3081,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Wire 1,datetime64[ns],DateTime,3055,"[1556881901000000000, 1556883970000000000, 155...",26,0.843882
Wire 2,datetime64[ns],DateTime,1079,"[None, 1556890326000000000, 155689199400000000...",2002,64.978903
Wire 3,datetime64[ns],DateTime,63,"[None, 1556945273000000000, 155694840800000000...",3018,97.955209
Wire 4,datetime64[ns],DateTime,14,"[None, 1557218903000000000, 155722409500000000...",3067,99.545602
Wire 5,datetime64[ns],DateTime,1,"[None, 1565280067000000000]",3080,99.967543
Wire 6,datetime64[ns],DateTime,73,"[None, 1557218240000000000, 155722359500000000...",3008,97.630639
Wire 7,datetime64[ns],DateTime,11,"[None, 1563885720000000000, 156388372400000000...",3070,99.642973
Wire 8,datetime64[ns],DateTime,19,"[None, 1557792212000000000, 155779419900000000...",3062,99.383317
Wire 9,datetime64[ns],DateTime,29,"[None, 1556986151000000000, 155935592600000000...",3052,99.058747


################################################## Summary for wire dataset ##################################################


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3081,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Wire 1,float64,Numerical,2251,"[60.059998, 96.052315, 91.160157, 89.063515, 8...",26,0.843882
Wire 2,float64,Numerical,713,"[nan, 9.11456, 9.143681, 12.376, 52.416003, 37...",2002,64.978903
Wire 3,float64,Numerical,56,"[nan, 250.041794, 276.324066, 96.047951, 88.64...",3018,97.955209
Wire 4,float64,Numerical,14,"[nan, 24.148801, 43.065361, 33.182243, 103.034...",3067,99.545602
Wire 5,float64,Numerical,1,"[nan, 15.132]",3080,99.967543
Wire 6,float64,Numerical,69,"[nan, 60.094322000000005, 54.122643, 64.212723...",3008,97.630639
Wire 7,float64,Numerical,10,"[nan, 5.035472, 8.49004, 1.053936, 0.234208, 1...",3070,99.642973
Wire 8,float64,Numerical,13,"[nan, 46.002319, 46.094879, 46.187439000000005...",3062,99.383317
Wire 9,float64,Numerical,25,"[nan, 35.0532, 4.622800000000001, 35.0168, 10....",3052,99.058747


Числовые распределения:

In [5]:
for dataset in raw.keys():
    print('#' * 50, 'Summary for', dataset, 'dataset', '#' * 50)
    display(round(raw[dataset].describe().T, 2))

################################################## Summary for arc dataset ##################################################


,count,mean,std,min,25%,50%,75%,max
key,14876.0,1615.22,934.57,1.00,806.00,1617.00,2429.00,3241.00
Активная мощность,14876.0,0.67,0.41,0.03,0.40,0.56,0.86,3.73
Реактивная мощность,14876.0,0.45,5.88,-715.50,0.29,0.42,0.64,2.68


################################################## Summary for bulk_t dataset ##################################################


,count,mean,std,min,25%,50%,75%,max
key,3129.0,1624.38,933.34,1.0,816.0,1622.0,2431.0,3241.0


################################################## Summary for bulk dataset ##################################################


...

,count,mean,std,min,25%,50%,75%,max
key,15907.0,1607.88,942.21,1.0,790.0,1618.0,2427.0,3241.0
Температура,13006.0,1591.84,21.38,1191.0,1581.0,1591.0,1601.0,1705.0


################################################## Summary for wire_t dataset ##################################################


,count,mean,std,min,25%,50%,75%,max
key,3081.0,1623.43,933.0,1.0,823.0,1619.0,2434.0,3241.0


################################################## Summary for wire dataset ##################################################


,count,mean,std,min,25%,50%,75%,max
key,3081.0,1623.43,933.00,1.00,823.00,1619.00,2434.00,3241.00
Wire 1,3055.0,100.90,42.01,1.92,72.12,100.16,126.06,330.31
Wire 2,1079.0,50.58,39.32,0.03,20.19,40.14,70.23,282.78
Wire 3,63.0,189.48,99.51,0.14,95.14,235.19,276.25,385.01
Wire 4,14.0,57.44,28.82,24.15,40.81,45.23,76.12,113.23
Wire 5,1.0,15.13,NaN,15.13,15.13,15.13,15.13,15.13
Wire 6,73.0,48.02,33.92,0.03,25.05,42.08,64.21,180.45
Wire 7,11.0,10.04,8.61,0.23,6.76,9.02,11.89,32.85
Wire 8,19.0,53.63,16.88,45.08,46.09,46.28,48.09,102.76
Wire 9,29.0,34.16,19.93,4.62,22.06,30.07,43.86,90.05


И детальный отчет:

In [6]:
for dataset in raw.keys():
    print('#' * 50, 'Summary for', dataset, 'dataset', '#' * 50)
    ProfileReport(raw[dataset]).to_widgets()

################################################## Summary for arc dataset ##################################################


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

################################################## Summary for bulk_t dataset ##################################################


Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

...

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

################################################## Summary for wire_t dataset ##################################################


Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

################################################## Summary for wire dataset ##################################################


Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

**Выводы на основе предварительного анализа:**

Данные о каждой партии разбиты по датасетам. Всего собрана информация о 3239 партий. Названия столбцов встречаются на русском и английском языках. Их надо будет привести к формату `snake_case`. Некоторым столбцам назначен неправильный тип данных. Во всех датасетах присутствует столбец `key`, содержащий номер партии.

1. В датасете `data_arc.csv` 14876 записей о нагреве сплава. Явное дублирование и пропуски - отсутствуют. Столбцам `Начало нагрева дугой` и `Конец нагрева дугой` необходимо сменить тип данных на `datetime`. Название столбцов требуется перевести на английский язык.
   
2. В датасете `data_bulk.csv` 3129 сплава и записи о добавленных в них сыпучих присадок. Всего существует 15 присадок. Явное дублирование - отсутствует. Пропуски означают, что данная присадка не добавлялась в сплав. Пропуски будут заменены значением `0.0`.
   
3. В датасете `data_bulk_time.csv` аналогичные с предыдущим пунктом замечания. Дополнительно необходимо сменить тип данных на `datetime`.
   
4. В датасете `data_gas.csv` 3239 записей о подаче газа в сплавы. Явное дублирование и пропуски - отсутствуют. Название столбца `Газ 1` требуется перевести на английский язык.
   
5. В датасете `data_temp.csv` 18092 замеров температур. Явное дублирование - отсутствует. В столбце `Температура` встречаются пропуски. К сожалению, эти строки придется удалить, т.к. нет возможности восстановить пропущенные значения. Для столбца `Время замера` необходимо сменить тип данных на `datetime`. Название столбцов требуется перевести на английский язык.
   
6. Датасеты `data_wire.csv` и `data_wire_time.csv` содержат 3081 сплав и записи о добавленных в них проволочных присадок. Остальные замечания аналогичны как для датасетов `data_bulk.csv` и `data_bulk_time.csv`.

## Предобработка данных

Пройдем по каждому датасету и сделаем необходимые преобразования.

### Датасет `arc`

Зададим новые названия колонок, уберем странные значения, создадим сводную таблицу.

In [73]:
df = {}

df['arc'] = (
    raw['arc']
    .copy()
    .set_axis(
        ['key', 'heating_start', 'heating_end', 'active_power', 'reactive_power'], axis=1
    )
    .loc[lambda df:
        df.reactive_power >= 0
    ]
    .assign(
        heating_time_sec=lambda df: (df.heating_end - df.heating_start).dt.seconds
    )
    .pivot_table(
        index='key',
        values=['active_power', 'reactive_power', 'heating_time_sec'],
        aggfunc={
            'active_power': 'sum', 'reactive_power': 'sum',
            'heating_time_sec': ['sum', 'count']
        }
    )
    .reset_index()
    .set_axis(
        ['key', 'active_power', 'cycles_no', 'heating_time_sec', 'reactive_power'], axis=1
    )
    .loc[:, [
        'key', 'cycles_no', 'heating_time_sec', 'active_power', 'reactive_power'
    ]]
)

display(df['arc'].head(3))

,key,cycles_no,heating_time_sec,active_power,reactive_power
0,1,5,1098,4.878147,3.183241
1,2,4,811,3.052598,1.998112
2,3,5,655,2.525882,1.599076


### Датасет `bulk`

Зададим названия колонок, уберем неинформативные колонки, заполним пропуски.

In [72]:
df['bulk'] = (
    raw['bulk']
    .copy()
    .rename(
        columns=lambda column: snakecase(column) if column == 'key' else snakecase(column) + '_vol'
    )
    .drop(['bulk_8_vol'], axis=1)
    .fillna(0)
)

display(df['bulk'].head(3))

,key,bulk_1_vol,bulk_2_vol,bulk_3_vol,bulk_4_vol,bulk_5_vol,bulk_6_vol,bulk_7_vol,bulk_9_vol,bulk_10_vol,bulk_11_vol,bulk_12_vol,bulk_13_vol,bulk_14_vol,bulk_15_vol
0,1,0.0,0.0,0.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,206.0,0.0,150.0,154.0
1,2,0.0,0.0,0.0,73.0,0.0,0.0,0.0,0.0,0.0,0.0,206.0,0.0,149.0,154.0
2,3,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,205.0,0.0,152.0,153.0


### Датасет `bulk_t`

Зададим названия колонок, уберем неинформативные колонки.

In [91]:
df['bulk_t'] = (
    raw['bulk_t']
    .copy()
    .rename(
        columns=lambda column: snakecase(column) if column == 'key' else snakecase(column) + '_dt'
    )
    .drop(['bulk_8_dt'], axis=1)
    # .fillna(0)
)

display(df['bulk_t'].head(3))

,key,bulk_1_dt,bulk_2_dt,bulk_3_dt,bulk_4_dt,bulk_5_dt,bulk_6_dt,bulk_7_dt,bulk_9_dt,bulk_10_dt,bulk_11_dt,bulk_12_dt,bulk_13_dt,bulk_14_dt,bulk_15_dt
0,1,NaT,NaT,NaT,2019-05-03 11:21:30,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 11:03:52,NaT,2019-05-03 11:03:52,2019-05-03 11:03:52
1,2,NaT,NaT,NaT,2019-05-03 11:46:38,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 11:40:20,NaT,2019-05-03 11:40:20,2019-05-03 11:40:20
2,3,NaT,NaT,NaT,2019-05-03 12:31:06,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 12:09:40,NaT,2019-05-03 12:09:40,2019-05-03 12:09:40


### Датасет `gas`

Зададим названия колонок.

In [92]:
df['gas'] = (
    raw['gas']
    .copy()
    .set_axis(['key', 'gas'], axis=1)
)

display(df['gas'].head(3))

,key,gas
0,1,29.749986
1,2,12.555561
2,3,28.554793


### Датасет `temp`

Зададим новые названия колонок, создадим сводную таблицу.

In [106]:
df['temp'] = (
    raw['temp']
    .copy()
    .set_axis(['key', 'dt', 'temp_degc'], axis=1)
    .sort_values(by=['key', 'dt'], ascending=True)
    .pivot_table(
        index='key',
        values=['dt', 'temp_degc'],
        aggfunc={'dt': ['first', 'last', 'count'], 'temp_degc': ['first', 'last']}
    )
    .reset_index()
    .set_axis(['key', 'probe_count', 'dt_start', 'dt_end', 'temp_start_degc', 'temp_end_degc'], axis=1)
    .loc[:, [
        'key', 'probe_count', 'temp_start_degc', 'temp_end_degc'
    ]]
)

display(df['temp'].head(3))

,key,probe_count,temp_start_degc,temp_end_degc
0,1,5,1571.0,1613.0
1,2,6,1581.0,1602.0
2,3,5,1596.0,1599.0


### Датасет `wire`

Зададим названия колонок, уберем неинформативные колонки.

In [107]:
df['wire'] = (
    raw['wire']
    .copy()
    .rename(
        columns=lambda column: snakecase(column) if column == 'key' else snakecase(column) + '_vol'
    )
    .drop(['wire_5_vol'], axis=1)
    .fillna(0)
)

display(df['wire'].head(3))

,key,wire_1_vol,wire_2_vol,wire_3_vol,wire_4_vol,wire_6_vol,wire_7_vol,wire_8_vol,wire_9_vol
0,1,60.059998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,96.052315,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,91.160157,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Датасет `wire_t`

Зададим названия колонок, уберем неинформативные колонки.

In [109]:
df['wire_t'] = (
    raw['wire_t']
    .copy()
    .rename(
        columns=lambda column: snakecase(column) if column == 'key' else snakecase(column) + '_dt'
    )
    .drop(['wire_5_dt'], axis=1)
    # .fillna(0)
)

display(df['wire_t'].head(3))

,key,wire_1_dt,wire_2_dt,wire_3_dt,wire_4_dt,wire_6_dt,wire_7_dt,wire_8_dt,wire_9_dt
0,1,2019-05-03 11:11:41,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,2,2019-05-03 11:46:10,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,3,2019-05-03 12:13:47,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [9]:
# df = raw.copy()

# df['arc'].columns = [
#     'key', 'heating_start', 'heating_end', 'active_power', 'reactive_power'
# ]
# df['bulk'].columns = [
#     'key', 'bulk_1', 'bulk_2', 'bulk_3', 'bulk_4', 'bulk_5', 'bulk_6', 'bulk_7',
#     'bulk_8', 'bulk_9', 'bulk_10', 'bulk_11', 'bulk_12', 'bulk_13', 'bulk_14', 'bulk_15'
# ]
# df['bulk_t'].columns = [
#     'key', 'bulk_1', 'bulk_2', 'bulk_3', 'bulk_4', 'bulk_5', 'bulk_6', 'bulk_7', 
#     'bulk_8', 'bulk_9', 'bulk_10', 'bulk_11', 'bulk_12', 'bulk_13', 'bulk_14', 'bulk_15'
# ]
# df['gas'].columns = [
#     'key', 'gas'
# ]
# df['temp'].columns = [
#     'key', 'time', 'temp'
# ]
# df['wire'].columns = [
#     'key', 'wire_1', 'wire_2', 'wire_3', 'wire_4', 'wire_5', 'wire_6', 'wire_7', 'wire_8', 'wire_9'
# ]
# df['wire_t'].columns = [
#     'key', 'wire_1', 'wire_2', 'wire_3', 'wire_4', 'wire_5', 'wire_6', 'wire_7', 'wire_8', 'wire_9'
# ]

Пропущенные значения.